In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# import os
# # Set the desired working directory
# new_working_directory = '/content/drive/My Drive'

# # Change the current working directory
# os.chdir(new_working_directory)

# Import dataset

In [2]:
import pandas as pd
import numpy as np
from scipy import special
from scipy import stats
import matplotlib.pyplot as plt
import math
import itertools
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)

In [3]:
df = pd.read_csv('2.22 combined dataset of stock data and balance sheet data.csv')
df['isin'].nunique()

199

# Change fiscal_quarter

In [4]:
series_list = []
for i in range(1,(df[df['company_name'] == df['company_name'].unique()[0]].shape[0])+1):
  series_list.append(i)

In [5]:
temp = pd.DataFrame({'fiscal_quarter': series_list * (len(df['company_name'].unique()))})
df['fiscal_quarter'] = temp['fiscal_quarter']

In [6]:
len(df['company_name'].unique())

199

In [7]:
temp['fiscal_quarter'].shape

(14328,)

In [8]:
df["datadate"] = pd.to_datetime(df["datadate"])

In [9]:
df.head(5)

,datadate,key,fiscal_quarter,fiscal_year,total_assets,total_liabilities,isin,company_name,gic_industries,iso_country_code,industry_name,is_finance,Open,High,Low,Close,Volume
0,2006-03-31,212935.0,1,2006,74.4419,27.4910,ARAGRO010015,AGROMETAL SA,201060.0,ARG,Machinery,False,0.185858,0.202586,0.170246,0.185858,8522223.00
1,2006-06-30,212935.0,2,2006,77.1485,32.3303,ARAGRO010015,AGROMETAL SA,201060.0,ARG,Machinery,False,0.185858,0.187717,0.131959,0.141996,8754235.25
2,2006-09-30,212935.0,3,2006,89.3972,36.4684,ARAGRO010015,AGROMETAL SA,201060.0,ARG,Machinery,False,0.142368,0.148687,0.133818,0.146456,6738297.00
3,2006-12-31,212935.0,4,2006,77.2313,23.2864,ARAGRO010015,AGROMETAL SA,201060.0,ARG,Machinery,False,0.147943,0.174707,0.144970,0.162440,8879853.35
4,2007-03-31,212935.0,5,2007,83.2715,29.5663,ARAGRO010015,AGROMETAL SA,201060.0,ARG,Machinery,False,0.160571,0.179738,0.152226,0.158462,8679271.40


# Define functions used to implement Zhou's Model using emperical data from company balance sheet and stock returns

In [10]:
### These are function producing the estimated parameters used in the Zhou(2001) model
def get_companies():
  return df['company_name'].unique()

def get_industries():
  return df['industry_name'].unique()

def get_date():
  return df['datadate'].unique()

def get_num_quarter():
  return len(df['fiscal_quarter'].value_counts())

def get_country_codes():
  return df['iso_country_code'].unique()

def get_companies_from_industry(industry):
  return df[df['industry_name'] == industry]['company_name'].unique()

def get_companies_from_country(country):
  return df[df['iso_country_code'] == country]['company_name'].unique()

def get_finance_companies(country):
  return df[(df['is_finance']==True)&(df['iso_country_code'] == country)]['company_name'].unique().tolist()
def get_non_finance_companies(country):
  return df[(df['is_finance']==False)&(df['iso_country_code'] == country)]['company_name'].unique().tolist()
    

def get_Vi0(start,company1,company2):
  '''
  This function takes the a start date and two company names to return the starting value of assets
  Output: [v_company1_0,v_company_2_0]
  '''
  v_10 = df[(df['company_name'] == company1)&(df['datadate'] == start)]['total_assets'].item()
  v_20 = df[(df['company_name'] == company2)&(df['datadate'] == start)]['total_assets'].item()
  return v_10,v_20

def get_Ki(start,company1,company2):
  '''
  This function takes the a start date and two company names to return the starting value of Ki := short-term liability + 50% long-term liability
                                                                                                 = total liability -50% long-term liability
  Output: k_company1,k_company_2
  '''
  total_liability_1 = df.loc[(df['company_name'] == company1)&(df['datadate'] == start),['total_liabilities']].iloc[0,0]
  total_liability_2 = df.loc[(df['company_name'] == company2)&(df['datadate'] == start),['total_liabilities']].iloc[0,0]
  # long_term_liability_1 = df.loc[(df['company_name'] == company1)&(df['datadate'] == start),['long_term_liabilities']].iloc[0,0]
  # long_term_liability_2 = df.loc[(df['company_name'] == company2)&(df['datadate'] == start),['long_term_liabilities']].iloc[0,0]
  return  0.75*total_liability_1,0.75*total_liability_2

def get_t(start,end):
  '''
  This function takes the start date and end date of a time horizon and return t
  in Zhou(2001) formula
  Output: t(in years)
  '''
  start_idx = np.where(df['datadate'] == start)[0][0]
  end_idx = np.where(df['datadate'] == end)[0][0]
  return (end_idx - start_idx)/4

def get_sigma_i(start,end,company):
  '''
  This function returns the estimated asset volatility of a firm
  '''
  close_list = df[(df['datadate'] >= start) & (df['datadate'] <= end)&(df['company_name'] == company)]['Close']
  close_pct = close_list.pct_change()[1:]
  #print(close_list)
  return close_pct.std()


def get_rho(start,end,companies,column,company_1 = None,company_2 = None):
  '''
  This function takes the start date and end date of a time frame and returns the correlation matrix of asset value in a given time horizon
  Note companies is a numpy array containing the name of companies
  Output: A list object:[whole correlation matrix, corr(companyname1,companyname2)] if company_1 and company_2 specified
          the whole correlation matrix otherwise
  '''
  diff_matrix = np.zeros((int(get_t(start,end)*4),len(companies)))
  for i in [i for i in range(0,len(companies))]:
    col_val = np.diff(np.log(df[(df['datadate'] >= start) & (df['datadate'] <= end)&(df['company_name'] == companies[i])][column]))
    diff_matrix [:,i]= col_val
  diff_matrix = pd.DataFrame(diff_matrix)
  corr_df  = diff_matrix.corr()
  if company_1 == None and company_2 == None:
    return corr_df
  else:
    company_1_idx = companies.tolist().index(company_1)
    company_2_idx = companies.tolist().index(company_2)
    output = corr_df[company_1_idx][company_2_idx]
    return output

In [11]:
get_finance_companies('ARG')
# df[(df['is_finance']==True)&(df['iso_country_code'] == get_country_codes()[0])]['company_name'].unique().tolist()


['BANCO MACRO SA',
 'BANCO HIPOTECARIO SA',
 'BANCO BBVA ARGENTINA SA',
 'GRUPO FINANCIERO GALICIA SA']

In [12]:
for i in range(len(get_country_codes())):
  print(get_country_codes()[i], df[df['iso_country_code'] == get_country_codes()[i]]['company_name'].nunique())

ARG 26
PER 1
CUW 1
BRA 54
CHL 51
COL 10
JAM 1
CYM 1
MEX 48
PAN 2
TTO 1
VGB 3


In [13]:
df[df['iso_country_code']== 'COL']

,datadate,key,fiscal_quarter,fiscal_year,total_assets,total_liabilities,isin,company_name,gic_industries,iso_country_code,industry_name,is_finance,Open,High,Low,Close,Volume
9576,2006-03-31,205182.0,1,2006,1.820810e+05,38980.180000,COC07PA00027,MINEROS SA,151040.0,COL,Metals & Mining,False,0.593468,0.784226,0.572273,0.741836,2.208212e+07
9577,2006-06-30,205182.0,2,2006,1.820810e+05,38980.180000,COC07PA00027,MINEROS SA,151040.0,COL,Metals & Mining,False,0.746075,1.038570,0.678250,0.792704,4.058738e+07
9578,2006-09-30,205182.0,3,2006,1.820810e+05,38980.180000,COC07PA00027,MINEROS SA,151040.0,COL,Metals & Mining,False,0.792704,0.932593,0.763031,0.801182,5.809771e+06
9579,2006-12-31,205182.0,4,2006,1.820810e+05,38980.180000,COC07PA00027,MINEROS SA,151040.0,COL,Metals & Mining,False,0.792704,1.059765,0.775748,0.996179,7.799280e+06
9580,2007-03-31,205182.0,5,2007,1.841106e+05,35910.398356,COC07PA00027,MINEROS SA,151040.0,COL,Metals & Mining,False,1.131413,1.169929,1.015865,1.145857,1.009397e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10291,2022-12-31,292768.0,68,2022,2.580020e+06,584876.000000,COW19PA00020,PROTECCION SA,402030.0,COL,Capital Markets,True,28.158742,28.158742,28.158742,28.158742,3.225492e+03
10292,2023-03-31,292768.0,69,2023,2.698882e+06,593777.147000,COW19PA00020,PROTECCION SA,402030.0,COL,Capital Markets,True,27.556860,27.556860,18.371240,18.371240,2.909312e+03
10293,2023-06-30,292768.0,70,2023,2.784071e+06,597825.375000,COW19PA00020,PROTECCION SA,402030.0,COL,Capital Markets,True,18.371240,18.371240,18.371240,18.371240,2.173139e+03
10294,2023-09-30,292768.0,71,2023,2.786187e+06,613564.366000,COW19PA00020,PROTECCION SA,402030.0,COL,Capital Markets,True,18.371240,18.371240,9.185620,9.185620,1.752061e+03


In [14]:
get_Vi0('2006-03-31','PAMPA ENERGIA SA','TELECOM ARGENTINA'	)

(36.4952, 8916.0)

In [15]:
get_Ki('2006-03-31','PAMPA ENERGIA SA','TELECOM ARGENTINA')

(6.87285, 5249.25)

In [16]:

get_t('2006-03-31','2021-03-31')

15.0

In [17]:
# s = get_rho('2018-03-31','2023-03-31',get_companies(),'Close').stack()
# l = list(filter( lambda x: x[0]!= x[1],s[s>0.99].index.tolist()))
# l

In [18]:
get_rho(get_date()[0],get_date()[3],get_companies(),'Close').head(3)

,0,1,2,3,4,5,6,7,8,9,...,189,190,191,192,193,194,195,196,197,198
0,1.000000,-0.633200,0.677572,0.904816,0.989767,0.721817,0.835530,0.495626,0.901614,-0.559971,...,0.838838,-0.315740,0.982783,0.674592,0.944351,0.904763,0.945420,0.653276,0.565793,0.986711
1,-0.633200,1.000000,0.140197,-0.243364,-0.516281,0.078610,-0.103794,0.358406,-0.236120,0.995832,...,-0.952497,0.934322,-0.765304,0.144200,-0.343368,-0.902549,-0.850847,-0.999656,-0.996450,-0.499024
2,0.677572,0.140197,1.000000,0.926237,0.775580,0.998081,0.970224,0.974593,0.929024,0.229915,...,0.168003,0.483899,0.530020,0.999992,0.881786,0.299800,0.400939,-0.114188,-0.223054,0.788068


In [19]:

def get_default_corr_zhou(v1,v2,k1,k2,sigma1,sigma2,rho,t):
  '''
  This function returns default correlation calculated by zhou
  '''
  z1 = (math.log(v1/k1))/sigma1
  z2 = (math.log(v2/k2))/sigma2
  alpha = math.atan((-math.sqrt(1-rho**2))/rho) if rho < 0 else math.pi +  math.atan((-math.sqrt(1-rho**2))/rho)
  theta0 = math.atan((z2*math.sqrt(1-rho**2))/(z1-rho*z2)) if (z2*math.sqrt(1-rho**2))/(z1-rho*z2) > 0 else math.pi + math.atan((z2*math.sqrt(1-rho**2))/(z1-rho*z2))
  r0 = z2/math.sin(theta0)
  inf_sum = 0
  for n in [i for i in range(1,100000,2)]:
    bassel_sum = special.iv(0.5*(((n*math.pi)/alpha)+1),(r0**2/(4*t)))+ special.iv(0.5*(((n*math.pi)/alpha)-1),(r0**2/(4*t)))
    inf_sum += 1/n * math.sin((n*math.pi*theta0)/alpha)*bassel_sum
  marginal_prob_1 =  2*stats.norm.cdf(-z1/math.sqrt(t)) if 2*stats.norm.cdf(-z1/math.sqrt(t))< 1 else 0.9
  marginal_prob_2 = 2*stats.norm.cdf(-z2/math.sqrt(t))if 2*stats.norm.cdf(-z2/math.sqrt(t))<1 else 0.9
  var_1 = marginal_prob_1*(1-marginal_prob_1)
  var_2 = marginal_prob_2*(1-marginal_prob_2)
  either_default_prob = 1-(2*r0/(math.sqrt(2*math.pi*t)))*math.exp(-(r0**2/(4*t)))*inf_sum
  default_corr = (marginal_prob_1+marginal_prob_2-either_default_prob-marginal_prob_1*marginal_prob_2)/(math.sqrt(var_1*var_2))
  return default_corr

def analyse_two_companies_zhou(start,end,companies,company1,company2,prob_sub = 0.9):
  '''
  This function returns the default correlation between 2 companies in a given time horizon by using the
  closed form formula from Zhou(2001)
  prob_sub is the subsituting probability if one of the marginal probability is greater than 1
  '''
  ### get the essential parameter for Zhou's either probability default
  # print(companies.tolist().index(company1),companies.tolist().index(company2))
  v1 = get_Vi0(start,company1,company2)[0]
  v2 = get_Vi0(start,company1,company2)[1]
  k1 = get_Ki(start,company1,company2)[0]
  k2 = get_Ki(start,company1,company2)[1]
  sigma1 = get_sigma_i(start,end,company1)
  sigma2 = get_sigma_i(start,end,company2)
  #print(sigma1,sigma2)
#   print(get_rho(start,end,companies,'Close',company1,company2))
  if (abs(get_rho(start,end,companies,'Close',company1,company2))>=0.99)or(get_rho(start,end,companies,'Close',company1,company2)==0):
    rho = 0.5
  else:
    rho = get_rho(start,end,companies,'Close',company1,company2)
#   print(rho)
  t = get_t(start,end)
  z1 = (math.log(v1/k1))/sigma1
  z2 = (math.log(v2/k2))/sigma2
  alpha = math.atan((-math.sqrt(1-rho**2))/rho) if rho < 0 else math.pi +  math.atan((-math.sqrt(1-rho**2))/rho)
  theta0 = math.atan((z2*math.sqrt(1-rho**2))/(z1-rho*z2)) if (z2*math.sqrt(1-rho**2))/(z1-rho*z2) > 0 else math.pi + math.atan((z2*math.sqrt(1-rho**2))/(z1-rho*z2))
  r0 = z2/math.sin(theta0)
  inf_sum = 0
  for n in [i for i in range(1,1000000,2)]:
    bassel_sum = special.iv(0.5*(((n*math.pi)/alpha)+1),(r0**2/(4*t)))+ special.iv(0.5*(((n*math.pi)/alpha)-1),(r0**2/(4*t)))
    inf_sum += 1/n * math.sin((n*math.pi*theta0)/alpha)*bassel_sum
  #print(round(inf_sum,1),round(v1/k1,1),round(v2/k2,1),round(rho,2),t,round(z1,1),round(z2,1),round(alpha,1),round(theta0,1),round(r0,1))
  ### calculate the marginal default probability and correlation
  marginal_prob_1 = 2*stats.norm.cdf(-z1/math.sqrt(t))
  marginal_prob_2 = 2*stats.norm.cdf(-z2/math.sqrt(t))
  ### store the original value if any of the marginal prob >=1
  marginal_prob_1_original = marginal_prob_1 if marginal_prob_1 >=1 else marginal_prob_1
  marginal_prob_2_original = marginal_prob_2 if marginal_prob_2 >=1 else marginal_prob_2
  ### use the prob_sub to calculate the correlation
  marginal_prob_1_cal = prob_sub if marginal_prob_1 >=1 else marginal_prob_1
  marginal_prob_2_cal = prob_sub if marginal_prob_2 >=1 else marginal_prob_2
  var_1 = marginal_prob_1*(1-marginal_prob_1_cal)
  var_2 = marginal_prob_2*(1-marginal_prob_2_cal)
  either_default_prob = 1-(2*r0/(math.sqrt(2*math.pi*t)))*math.exp(-(r0**2/(4*t)))*inf_sum
  ### if the inf_sum returns value NaN just put default correlation to 1.5 and so that it can be ignored in later calculation
#   print(np.isnan(get_default_corr_zhou(v1,v2,k1,k2,sigma1,sigma2,rho,t)))
  default_corr = (marginal_prob_1_cal+marginal_prob_2_cal-either_default_prob-marginal_prob_1_cal*marginal_prob_2_cal)/(math.sqrt(var_1*var_2)) if (np.isnan(get_default_corr_zhou(v1,v2,k1,k2,sigma1,sigma2,rho,t)) == False) and (rho !=0.5) else 1.5
  
  return default_corr,marginal_prob_1_original,marginal_prob_2_original


def get_decorr_two_group_companies_zhou(start,end,company_list1,company_list2,prob_sub=0.9,show_non_normal = None):
  companies = get_companies()
  if company_list1 != company_list2:
    combination_list =  list(itertools.product(company_list1,company_list2))
  else:
    combination_list = list(filter(lambda x: x[0]!=x[1],list(itertools.combinations(company_list1,2))))
  # print(len(company_list1),len(company_list2),len(combination_list))
#   print(company_list1)
#   print(company_list2)
#   print(combination_list)
  print(len(combination_list))
  num_pairs = len(combination_list)
  corr_sum = 0
  not_normal_sum = 0
  non_normal_dic = {}
  for (firm1,firm2) in combination_list:
    result = analyse_two_companies_zhou(start,end,companies,firm1,firm2,prob_sub)
    corr = result[0]
    #print(corr_sum)
    prob_default_1 = result[1]
    prob_default_2 = result[2]
    # print(corr,prob_default_1,prob_default_2)
    # print(abs(corr)>1)
#     print(corr,type(corr))
    if (abs(corr) > 1) :
      num_pairs -= 1
#       print(num_pairs)
      corr_sum += 0
      non_normal_dic[corr] = [prob_default_1,prob_default_2]
    else:
      corr_sum += corr
  #print(num_pairs)
  if show_non_normal == 'Yes':
    return non_normal_dic
  else:
    return corr_sum/num_pairs if num_pairs != 0 else None

In [20]:
import time
start = time.time()
analyse_two_companies_zhou('2021-12-31','2023-12-31',get_companies(),get_companies()[3],get_companies()[8],prob_sub = 0.9)
end = time.time()
end-start

3.3920137882232666

# get the country coefficient

In [21]:
def get_country_corr(start,end,country,prob_sub=0.9):
  companies = get_companies()
  company_list_financial = df[(df['iso_country_code'] == country)&(df['is_finance'] == True)]['company_name'].unique().tolist()
  company_list_non_financial = df[(df['iso_country_code'] == country)&(df['is_finance'] == False)]['company_name'].unique().tolist()
  if (len(company_list_financial) < 1) or (len(company_list_non_financial) < 1):
    return None
  else:
    return get_decorr_two_group_companies_zhou(start,end,company_list_financial,company_list_non_financial)

def get_country_corr_matrix(start,end,prob_sub=0.9):
  countries = get_country_codes()
  mat = pd.DataFrame({'country':countries,'country_corr':[0 for i in range(len(countries))]})
  for i in range(len(countries)):
    mat.at[i,'country_corr'] = get_country_corr(start,end,get_country_codes()[i])
#     print(mat)
  return mat

In [23]:
# get_country_corr('2018-12-31','2021-12-31','ARG')

In [24]:
# try:
#   print(get_country_corr('2018-12-31','2021-12-31','ARG'))
# except:
#   pass

In [22]:
analyse_two_companies_zhou('2022-12-31','2023-12-31',get_companies(),'GRUPO DE INVERSIONES SURAMER', 'MINEROS SA')

(-1.771871617782744e+90, 5.064646195530967e-191, 7.751865664855691e-23)

In [26]:
# get_country_corr('2022-12-31','2023-12-31','COL')

In [27]:
# print(get_country_corr('2022-12-31','2023-12-31','VGB'))

In [25]:
get_country_corr_matrix('2022-12-31','2023-12-31')

88
405
308
24
320
1
2


,country,country_corr
0,ARG,0.328526
1,PER,NaN
2,CUW,NaN
3,BRA,0.026413
4,CHL,-0.058334
5,COL,0.037521
6,JAM,NaN
7,CYM,NaN
8,MEX,0.031872
9,PAN,0.312606


In [26]:
get_country_corr_matrix('2021-12-31','2023-12-31')

88
405
308
24
320
1
2


,country,country_corr
0,ARG,0.395769
1,PER,NaN
2,CUW,NaN
3,BRA,0.042270
4,CHL,0.005728
5,COL,0.017695
6,JAM,NaN
7,CYM,NaN
8,MEX,0.023148
9,PAN,0.394991


In [ ]:
get_country_corr_matrix('2018-12-31','2023-12-31')

88
405
308
24
320


# Calculate the financial sector coefficient


In [ ]:
list(filter(lambda x: x[0]!=x[1],list(itertools.combinations(['a','b','c','d'],2))))

In [23]:
def get_finance_corr(start,end,country,prob_sub=0.9):
  companies = get_companies()
  company_list_financial = df[(df['iso_country_code'] == country)&(df['is_finance'] == True)]['company_name'].unique().tolist()
  if (len(company_list_financial) <= 1):
    return None
  else:
    return get_decorr_two_group_companies_zhou(start,end,company_list_financial,company_list_financial)

In [24]:
def get_finance_corr_matrix(start,end,prob_sub=0.9):
  countries = get_country_codes()
  mat = pd.DataFrame({'country':countries,'finance_corr':[0 for i in range(len(countries))]})
  for i in range(len(countries)):
    mat.at[i,'finance_corr'] = get_finance_corr(start,end,get_country_codes()[i])
  return mat

In [25]:
# get_finance_corr('2018-12-31','2023-12-31','ARG')

In [50]:
print(get_finance_corr_matrix('2022-12-31','2023-12-31'))

6
36
21
6
28
   country  finance_corr
0      ARG      0.640718
1      PER           NaN
2      CUW           NaN
3      BRA      0.025693
4      CHL      0.000101
5      COL      0.111406
6      JAM           NaN
7      CYM           NaN
8      MEX      0.018942
9      PAN           NaN
10     TTO           NaN
11     VGB           NaN


In [51]:
get_finance_corr_matrix('2021-12-31','2023-12-31')

6
36
21
6
28


,country,finance_corr
0,ARG,0.648866
1,PER,NaN
2,CUW,NaN
3,BRA,0.051492
4,CHL,0.007554
5,COL,0.027422
6,JAM,NaN
7,CYM,NaN
8,MEX,0.024912
9,PAN,NaN


In [47]:
get_finance_corr_matrix('2018-12-31','2023-12-31')

6


KeyboardInterrupt: 

In [ ]:

get_finance_corr_matrix('2013-12-31','2023-12-31')

In [ ]:
get_finance_corr_matrix('2006-12-31','2023-12-31')

# Calculate the non financial sector coefficient


In [ ]:
def get_non_finance_corr(start,end,country,prob_sub=0.9):
  companies = get_companies()
  company_list_non_financial = df[(df['iso_country_code'] == country)&(df['is_finance'] == False)]['company_name'].unique().tolist()
  if (len(company_list_non_financial) <= 1):
    return None
  else:
    return get_decorr_two_group_companies_zhou(start,end,company_list_non_financial,company_list_non_financial)

In [ ]:
def get_non_finance_corr_matrix(start,end,prob_sub=0.9):
  countries = get_country_codes()
  mat = pd.DataFrame({'country':countries,'non_finance_corr':[0 for i in range(len(countries))]})
  for i in range(len(countries)):
    mat.at[i,'non_finance_corr'] = get_non_finance_corr(start,end,get_country_codes()[i])
  return mat

In [ ]:
# get_non_finance_corr('2018-12-31','2023-12-31','COL',prob_sub=0.9)

In [ ]:
get_non_finance_corr_matrix('2022-12-31','2023-12-31')


In [ ]:
get_non_finance_corr_matrix('2021-12-31','2023-12-31')

In [ ]:
get_non_finance_corr_matrix('2018-12-31','2023-12-31')

In [ ]:

get_non_finance_corr_matrix('2006-12-31','2023-12-31')

In [33]:
df.head()

,datadate,key,fiscal_quarter,fiscal_year,total_assets,total_liabilities,isin,company_name,gic_industries,iso_country_code,industry_name,is_finance,Open,High,Low,Close,Volume
0,2006-03-31,212935.0,1,2006,74.4419,27.4910,ARAGRO010015,AGROMETAL SA,201060.0,ARG,Machinery,False,0.185858,0.202586,0.170246,0.185858,8522223.00
1,2006-06-30,212935.0,2,2006,77.1485,32.3303,ARAGRO010015,AGROMETAL SA,201060.0,ARG,Machinery,False,0.185858,0.187717,0.131959,0.141996,8754235.25
2,2006-09-30,212935.0,3,2006,89.3972,36.4684,ARAGRO010015,AGROMETAL SA,201060.0,ARG,Machinery,False,0.142368,0.148687,0.133818,0.146456,6738297.00
3,2006-12-31,212935.0,4,2006,77.2313,23.2864,ARAGRO010015,AGROMETAL SA,201060.0,ARG,Machinery,False,0.147943,0.174707,0.144970,0.162440,8879853.35
4,2007-03-31,212935.0,5,2007,83.2715,29.5663,ARAGRO010015,AGROMETAL SA,201060.0,ARG,Machinery,False,0.160571,0.179738,0.152226,0.158462,8679271.40


## Get Global Coefficient

In [32]:
country_list = list(df['iso_country_code'].unique())
company_country_dict = {}
for country in country_list:
    company_country_dict[country] = [get_finance_companies(country), get_non_finance_companies(country)]

non_financial_list = []
for country in country_list:
    non_financial_list.append(company_country_dict[country][1])

pair_list = []
for i in range(len(country_list)):
    non_financial_list_copy = non_financial_list.copy()
    non_financial_list_copy.pop(i)
    original_list = non_financial_list_copy
    flattened_list = [item for sublist in original_list for item in sublist]
    pair_list += list(itertools.product(company_country_dict[country_list[i]][0], flattened_list))


In [42]:
def get_global_coefficient(start,end):
    corr_sum = 0
    num_pair = len(pair_list)
    for firm1,firm2 in pair_list:
        increment = analyse_two_companies_zhou(start,end,get_companies(),firm1,firm2)[0]
        print(increment)
        if abs(increment) < 1:
            corr_sum += increment
        else:
            corr_sum += 0
            num_pair -=1
        print(corr_sum)
    print(num_pair)
    return corr_sum/num_pair
        
    

In [43]:
get_global_coefficient('2022-12-31','2023-12-31')

1.580204265285557e-07
1.580204265285557e-07
0.12330212846979119
0.12330228649021771
0.438264509520712
0.5615667960109297
-0.00388155086657022
0.5576852451443595
-2.7207081367052535e-06
0.5576825244362228
0.0001022590000992243
0.557784783436322
0.0010858452680257789
0.5588706287043478
-0.07190321032141898
0.4869674183829288
0.45534771709499644
0.9423151354779252
0.44215297746295973
1.384468112940885
-697243.3892493101
1.384468112940885
6.00322717931905e-05
1.384528145212678
0.07745190389934693
1.461980049112025
-0.1667615677822792
1.2952184813297458
0.0001601213780613117
1.2953786027078071
55950278611220.86
1.2953786027078071
0.42522102900871933
1.7205996317165264
1.5
1.7205996317165264
-9.635845779970038e+26
1.7205996317165264
0.4479128938608775
2.168512525577404
-2.9779647087870787e-06
2.1685095476126954
0.4525118645695265
2.621021412182222
0.04038860199975719
2.661410014181979
1.5
2.661410014181979
-5.749162486599045e-06
2.6614042650194927
3.7670782571050246e-06
2.6614080320977496
0.

1.5
4.833561344398036
1.5
4.833561344398036
0.07189871596091457
4.90546006035895
769896825.1918418
4.90546006035895
-0.0038824501490981786
4.901577610209852
-0.00013406366641798115
4.901443546543434
0.08847350869432996
4.989917055237764
0.047307067463128245
5.037224122700892
-0.027815955914568126
5.009408166786324
2.311445038237537e-05
5.009431281236706
1.3042371975640131e-05
5.009444323608681
4.947047158884874e-06
5.00944927065584
-0.013133008240554687
4.996316262415285
0.004737381800336827
5.001053644215622
24907282.16760538
5.001053644215622
-2.931980855638421e+24
5.001053644215622
6.758120464445762e-05
5.001121225420267
1.5
5.001121225420267
3207.957815904064
5.001121225420267
-0.07619281946723618
4.924928405953031
-2.169339150356942e-07
4.924928189019115
-1.8815464598767192e-05
4.924909373554517
0.0002357797276736436
4.92514515328219
-0.0044551040371169645
4.920690049245073
0.12911124588758552
5.049801295132659
2.5073493730610427e-06
5.049803802482033
-11842863696210.768
5.0498038

0.022641776789958094
11.36853748487949
0.27012085635356675
11.638658341233056
-0.018943872191848034
11.619714469041208
0.25188456664593956
11.871599035687147
0.013712428963298258
11.885311464650446
-0.002683112999631847
11.882628351650814
4.0813858487610333e-07
11.882628759789398
0.038428130681512636
11.921056890470911
0.01010157348036262
11.931158463951274
2.5105142911151508e+16
11.931158463951274
3.593358965082481e-05
11.931194397540924
3.5558275550867055e-07
11.93119475312368
3.7860224146458817e-06
11.931198539146095
8.531008716280925e-06
11.93120707015481
-4.85642037624102e-05
11.931158505951048
-0.040277073676510004
11.890881432274538
0.0013940183407562246
11.892275450615294
-175482582229.19098
11.892275450615294
4.874633162760703e-06
11.892280325248457
1.5
11.892280325248457
-0.0008853558552463331
11.891394969393211
-5429560.609465796
11.891394969393211
-0.005000281385977402
11.886394688007234
570492752423187.9
11.886394688007234
16241.881802223024
11.886394688007234
8.6837267264

1.5
12.751698209096636
0.5879659888901237
13.33966419798676
-498011671.55891895
13.33966419798676
0.0012401844513647016
13.340904382438126
0.00148619718694724
13.342390579625073
-0.010183852566360743
13.332206727058713
0.25865957235287806
13.59086629941159
-0.0032722044571676457
13.587594094954422
0.0001381674171480029
13.58773226237157
0.00039372278066598893
13.588125985152237
-7.467033824784907e-06
13.588118518118412
0.06505927662258786
13.653177794741
-0.001208134123545567
13.651969660617453
374636392.9402221
13.651969660617453
3.486261878836348e+24
13.651969660617453
0.0020535970826959875
13.65402325770015
1.5
13.65402325770015
48585.582904250645
13.65402325770015
-0.0024331616101437364
13.651590096090006
-1.4445141481308216e-08
13.651590081644864
-9.229685849911484e-07
13.651589158676279
0.0009285319829731833
13.652517690659252
-0.0001577959167349106
13.652359894742517
0.09149634098727696
13.743856235729794
-5.277154288425313e-06
13.743850958575505
-129068180569547.8
13.7438509585

0.47580840613655934
16.101643199054845
0.2712782921609587
16.372921491215802
0.14593723981867684
16.518858731034477
0.04328379332923524
16.56214252436371
0.2250018584309465
16.787144382794658
0.16619467382609318
16.953339056620752
0.24869945283367653
17.20203850945443
0.07423540516198557
17.276273914616414
0.04047179092739973
17.316745705543813
0.27602929216506045
17.592774997708872
0.24873847707336572
17.84151347478224
0.14683732555288626
17.988350800335127
0.31386286579376244
18.30221366612889
0.11568097705529976
18.417894643184187
0.12523043612007342
18.54312507930426
0.1584215186846644
18.701546597988926
0.07392005287122523
18.775466650860153
0.1896177258046303
18.965084376664784
0.19700444327545155
19.162088819940234
0.23250151439199165
19.394590334332225
0.09341296500643323
19.488003299338658
-1.8603624604464732e-06
19.488001438976198
2.915142420989638e+91
19.488001438976198
-0.00019681163397633223
19.48780462734222
1.5
19.48780462734222
-0.0010768109765717361
19.48672781636565
-

...

-5.485728974839635e-07
27.705195652251952
7.592564147584204e-05
27.705271577893427
0.001320253442801022
27.70659183133623
0.00021157369739908702
27.70680340503363
-1.1943678080125724e-05
27.70679146135555
0.00019251215835536928
27.706983973513907
0.0009219114326534588
27.70790588494656
-7.2497933167119884e+19
27.70790588494656
7.389669114790929e-06
27.707913274615674
0.0028274518942102587
27.710740726509883
0.004732991401961154
27.715473717911845
1.1296977693016977e-06
27.715474847609613
2.645868036223448e-07
27.715475112196415
808.9544540844572
27.715475112196415
0.011445777574964977
27.72692088977138
-4593255300987720.0
27.72692088977138
-3.6408292998595146
27.72692088977138
-3.365929776667789e-05
27.726887230473615
-4.4364752533153044e-07
27.72688678682609
-1.1216561486505098e-06
27.72688566516994
6.906115655008111e-07
27.726886355781506
1.0633439027503597e-06
27.726887419125408
0.2556285910424381
27.982516010167846
0.00015520164322137167
27.982671211811066
-4.170312979273668e-07
27

1.5
27.994665231141436
1.5
27.994665231141436
inf
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
inf
27.994665231141436
-inf
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
-inf
27.994665231141436
1.5
27.994665231141436
inf
27.994665231141436
-inf
27.994665231141436
-inf
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
inf
27.994665231141436
-inf
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
inf
27.994665231141436
inf
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
inf
27.994665231141436
-inf
27.994665231141436
1.5
27.994665231141436
inf
27.994665231141436
inf
27.994665231141436
inf
27.994665231141436
-inf
27.994665231141436
-inf
27.994665231141436
-inf
27.994665231141436
1.5
27.994665231141436
-inf
27.994665231141436
i

1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
inf
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.994665231141436
1.5
27.9946

1.5
38.4227476364823
1.5
38.4227476364823
1.5
38.4227476364823
-3.460724495669472e+79
38.4227476364823
1.5
38.4227476364823
-1.373124294019817e+80
38.4227476364823
-5.5683204361570006e+79
38.4227476364823
1.5
38.4227476364823
-6.862436764386444e+79
38.4227476364823
1.5
38.4227476364823
1.5
38.4227476364823
1.5
38.4227476364823
1.2572473219264153e+80
38.4227476364823
1.5
38.4227476364823
-4.079973470963941e+79
38.4227476364823
3.622160064315445e+79
38.4227476364823
-1.2345100432960484e+82
38.4227476364823
3.485517953336306e+88
38.4227476364823
5.7567976548710495e+82
38.4227476364823
-1.5343325951809967e+82
38.4227476364823
2.668064483055792e+80
38.4227476364823
-8.929467093069235e-96
38.4227476364823
1.5
38.4227476364823
6.99516446799453e+99
38.4227476364823
4.9600448000150494e+89
38.4227476364823
1.5
38.4227476364823
1.5
38.4227476364823
-6.87275705037553e+83
38.4227476364823
3.9292876040874646e+108
38.4227476364823
-6.540755430123309e-96
38.4227476364823
1.5
38.4227476364823
-6.767081

KeyboardInterrupt: 

In [ ]:
get_global_coefficient('2021-12-31','2023-12-31')

In [ ]:
get_global_coefficient('2018-12-31','2023-12-31')
    

In [ ]:
get_global_coefficient('2006-12-31','2023-12-31')
    

In [44]:
get_global_coefficient('2006-12-31','2008-12-31')

-5.2671636216391814e-06
-5.2671636216391814e-06
0.16415947502532732
0.16415420786170568
0.3265786576273161
0.49073286548902173
-0.008448635485903518
0.4822842300031182
0.007183538874953087
0.4894677688780713
-0.04873267935415652
0.44073508952391477
-1.4182904729764148e-05
0.440720906619185
0.057023326741552936
0.4977442333607379
0.010397050724996693
0.5081412840857347
-0.0037203448297134633
0.5044209392560212
0.1924648639861474
0.6968858032421685
0.00019224943692475565
0.6970780526790933
0.02402720565756158
0.7211052583366548
0.5024771544920523
1.2235824128287072
0.004685421895709003
1.2282678347244163
-0.019234058945896273
1.20903377577852
0.0338897892132583
1.2429235649917783
0.15357102010547047
1.3964945850972488
6.464552496006383e-07
1.3964952315524983
0.0008260090262893162
1.3973212405787876
0.0936156227587745
1.4909368633375621
0.054607868244135056
1.5455447315816973
0.02668214207304165
1.572226873654739
0.04355453579089615
1.615781409445635
0.302015650358621
1.917797059804256
-0

2.5981222771094354e-06
11.378308754577377
2.050543544073247e-06
11.378310805120922
0.0032464267279105606
11.381557231848833
4.5660214851830706e+73
11.381557231848833
1.5846244519392025e+83
11.381557231848833
962.87971692495
11.381557231848833
0.002220082637611067
11.383777314486444
1.5
11.383777314486444
-1146.670311053879
11.383777314486444
6.781628880200614e-05
11.383845130775246
0.12098390375925139
11.504829034534497
0.0013044469660516732
11.506133481500548
0.16791877695706536
11.674052258457614
-5.205948097445384e-08
11.674052206398134
7.854692772368006e-07
11.674052991867411
-81.45078235288483
11.674052991867411
6.716169732805482e+20
11.674052991867411
7.438525651809432e+30
11.674052991867411
0.0244223827783656
11.698475374645776
-0.0014869923519939235
11.696988382293782
-8.717440442600924e-06
11.69697966485334
-0.005897526552773703
11.691082138300565
0.0005791695825069639
11.691661307883072
0.0850012099272552
11.776662517810326
0.0008882639773836121
11.77755078178771
1.7759503473

-76.04875724080655
21.852484804005453
0.0008232710934779735
21.85330807509893
5.968421049360179e-06
21.85331404351998
0.31840394965984353
22.171717993179822
0.39982680201901327
22.571544795198836
-0.006598594324346103
22.56494620087449
0.02615762247495169
22.591103823349442
-0.045404134381733845
22.54569968896771
-1.1858620790517121e-06
22.54569850310563
0.19554853123696164
22.741247034342592
0.02224362475270025
22.763490659095293
-0.0013520155660363955
22.762138643529255
0.3253213780513399
23.087460021580593
0.00015675176006667199
23.08761677334066
0.06080449655920229
23.148421269899863
0.3709619625336609
23.519383232433523
0.0054340921029205754
23.524817324536443
0.07738430851450866
23.602201633050953
0.044983303434997055
23.64718493648595
0.1637283369924044
23.810913273478352
3.4544952695748476e-06
23.810916727973623
0.0014932531853580156
23.812409981158982
0.015337463078184344
23.827747444237165
0.10565280384951357
23.933400248086677
0.06813370503066934
24.001533953117345
0.1258581

0.06162481898050389
41.53710820619902
0.3444610535156673
41.88156925971469
4.278732476053103e-08
41.881569302502015
0.004604268788679997
41.8861735712907
7.69071415969244e-07
41.88617434036211
1.2165108988559739e-05
41.8861865054711
0.1835945762808836
42.069781081751984
0.036257244869536506
42.10603832662152
-5.815679659846716e-08
42.10603826846473
0.1866452183483199
42.29268348681305
5.392130457505145e+89
42.29268348681305
0.05612161539453531
42.348805102207585
1.9999511410706016e-05
42.348825101718994
0.0014001799521140403
42.35022528167111
5.133488181061852e-08
42.35022533300599
0.5940937384729978
42.944319071478986
0.024468381946965996
42.96878745342595
-2.997328680947218e+52
42.96878745342595
-1.1358003970934168e+33
42.96878745342595
-3.843015860544199e-05
42.96874902326735
0.020084061547374146
42.98883308481472
-1.7851349711956814e-05
42.98881523346501
5.674240857558841e-07
42.9888158008891
-3.8127133219611744e-07
42.988815419617765
2.500682622793923e-06
42.98881792030039
0.00407

0.033327998055933554
52.65881719883861
0.013340640117045645
52.672157838955656
0.012006141698504835
52.68416398065416
0.08613447964837335
52.770298460302534
0.062017539315518115
52.832315999618054
0.0002203643140025815
52.83253636393206
0.002061517608145194
52.834597881540205
0.0003925834857501021
52.83499046502595
-3.10814396042962e+20
52.83499046502595
0.21930223500088422
53.054292700026835
0.04666177815377908
53.10095447818061
0.09945828944077327
53.20041276762139
0.19153292559400206
53.39194569321539
0.20048559864400065
53.59243129185939
0.053804373163202086
53.646235665022594
0.00027831102393127666
53.64651397604653
0.3019398496902393
53.94845382573677
2.325847465393928e-06
53.94845615158424
0.03932748747427517
53.98778363905851
-4302904691055283.0
53.98778363905851
1.6672001445852741e-06
53.98778530625866
0.019447473232326238
54.007232779490984
9939654602322244.0
54.007232779490984
-424.5673331936756
54.007232779490984
0.04403479684101377
54.051267576331995
0.08373495992517348
54

...

0.038474264472498264
131.17691929837153
0.008080292142972284
131.1849995905145
-4.170714961222217e-06
131.18499541979955
1.281073867594157e-06
131.18499670087343
9296.450121647302
131.18499670087343
-1.7996481514636906e+23
131.18499670087343
2.2613668370646958e+33
131.18499670087343
0.0010206624167526349
131.18601736329018
0.1414325500961143
131.3274499133863
-0.006600832227919463
131.32084908115837
0.0003447724779794833
131.32119385363634
0.0004534328728133375
131.32164728650915
0.0005911386764076012
131.32223842518556
0.008106145238684297
131.33034457042424
-0.004604400187882162
131.32574017023637
0.00042575671867905134
131.32616592695504
0.002922311210024597
131.32908823816507
18344.237315532067
131.32908823816507
0.001577860620745377
131.33066609878583
26861065162.996597
131.33066609878583
0.04119753675443387
131.37186363554025
0.04463915245447731
131.41650278799472
-522409.4517735677
131.41650278799472
1.4852608103160816e-07
131.4165029365208
0.08172973755964826
131.49823267408044

-0.03945837648742394
147.4438959640626
0.05546910188131043
147.4993650659439
-7.000915232825492e-09
147.49936505894297
0.00802667792863681
147.5073917368716
2.648553715110366e-07
147.50739200172697
6.677763117846518e-06
147.50739867949008
0.03260569872640711
147.54000437821648
-0.04797580800253965
147.49202857021393
2.44298041635922e-07
147.49202881451197
0.15931774189755482
147.65134655640952
4.592349500357643e+90
147.65134655640952
0.043967673309347845
147.69531422971886
4.037955344785496e-05
147.6953546092723
0.004021994496251812
147.69937660376854
2.3013950136037178e-08
147.69937662678248
0.26870404544877924
147.96808067223125
-0.018033871170956155
147.9500468010603
4.512903827772099e+53
147.9500468010603
1.2911452700465311e+33
147.9500468010603
0.00021040745504405942
147.95025720851535
0.007667580781394409
147.95792478929675
-2.9746064802646803e-05
147.95789504323196
-4.3515681008129855e-06
147.95789069166386
1.408143730129672e-06
147.9578920998076
-9.516434366343806e-08
147.95789

0.02818600299288657
148.8188243994192
-0.0013709782928996148
148.81745342112632
0.04585925868755862
148.86331267981387
0.011554592853044039
148.8748672726669
0.00029006274058018783
148.8751573354075
0.000807371593618777
148.87596470700112
1.1199849967294644e-05
148.8759759068511
0.034173319418417954
148.9101492262695
0.0005328509352663091
148.91068207720477
0.04311472456726295
148.95379680177203
0.04460847008277566
148.9984052718548
-0.007399907613016006
148.9910053642418
0.00219372965860264
148.99319909390042
0.0021138429568718885
148.9953129368573
0.016198090608753078
149.01151102746604
0.032755647598887556
149.04426667506493
0.007762941070125933
149.05202961613506
0.010107728728134901
149.0621373448632
0.03042822939491253
149.09256557425812
0.02652684004964409
149.11909241430777
0.014454680086797277
149.13354709439457
0.18954757775075945
149.32309467214534
0.02891775877923149
149.3520124309246
0.004757909074414757
149.356770339999
0.01617492255620809
149.37294526255522
-0.0002555609

3.670636865247921e+98
150.3595488028913
1.2596628030598986e-05
150.35956139951935
-0.012690209922417808
150.34687118959692
0.0011328338771108809
150.34800402347403
31.856993038126962
150.34800402347403
-2.0525055029257266e-06
150.34800197096854
23054181.981911525
150.34800197096854
2.0016690683586771e+58
150.34800197096854
-7.659569835045515e+41
150.34800197096854
100855.14640472004
150.34800197096854
7.15149074236122e-05
150.34807348587597
-14250.712652508808
150.34807348587597
-345.12965767151013
150.34807348587597
-0.7658874937980162
149.58218599207797
0.21288885681864253
149.7950748488966
0.00038388286326491495
149.79545873175988
2.7496074195896145e+82
149.79545873175988
5.726187930246804e+92
149.79545873175988
331985733.967713
149.79545873175988
-0.0004449817693131907
149.79501374999057
1.5
149.79501374999057
148042189259.0908
149.79501374999057
0.03571134611101097
149.8307250961016
-6.485203928552726e-06
149.83071861089766
1.2470235799359687e-05
149.83073108113345
1.7059714251634

0.04289544521038898

In [ ]:
# def analyse_two_companies_zhou(start,end,companies,company1,company2,prob_sub=0.9):
#   '''
#   This function returns the default correlation between 2 companies in a given time horizon by using the
#   closed form formula from Zhou(2001)
#   prob_sub is the subsituting probability if one of the marginal probability is greater than 1
#   '''
#   ### get the essential parameter for Zhou's either probability default
#   # print(companies.tolist().index(company1),companies.tolist().index(company2))
#   v1 = get_Vi0(start,company1,company2)[0]
#   v2 = get_Vi0(start,company1,company2)[1]
#   k1 = get_Ki(start,company1,company2)[0]
#   k2 = get_Ki(start,company1,company2)[1]
#   sigma1 = get_sigma_i(company1)
#   sigma2 = get_sigma_i(company2)
#   rho = get_rho(start,end,companies,'Close',company1,company2)
#   t = get_t(start,end)
#   z1 = (math.log(v1/k1))/sigma1
#   z2 = (math.log(v2/k2))/sigma2
#   alpha = math.atan((-math.sqrt(1-rho**2))/rho) if rho < 0 else math.pi +  math.atan((-math.sqrt(1-rho**2))/rho)
#   theta0 = math.atan((z2*math.sqrt(1-rho**2))/(z1-rho*z2)) if (z2*math.sqrt(1-rho**2))/(z1-rho*z2) > 0 else math.pi + math.atan((z2*math.sqrt(1-rho**2))/(z1-rho*z2))
#   r0 = z2/math.sin(theta0)
#   inf_sum = 0
#   for n in [i for i in range(1,1000000,2)]:
#     bassel_sum = sp.special.iv(0.5*(((n*math.pi)/alpha)+1),(r0**2/(4*t)))+ sp.special.iv(0.5*(((n*math.pi)/alpha)-1),(r0**2/(4*t)))
#     inf_sum += 1/n * math.sin((n*math.pi*theta0)/alpha)*bassel_sum
#   # print(round(inf_sum,1),round(v1/k1,1),round(v2/k2,1),round(rho,2),t,round(z1,1),round(z2,1),round(alpha,1),round(theta0,1),round(r0,1))
#   ### calculate the marginal default probability and correlation
#   marginal_prob_1 = 2*sp.stats.norm.cdf(-z1/math.sqrt(t))
#   marginal_prob_2 = 2*sp.stats.norm.cdf(-z2/math.sqrt(t))
#   ### store the original value if any of the marginal prob >=1
#   marginal_prob_1_original = marginal_prob_1 if marginal_prob_1 >=1 else marginal_prob_1
#   marginal_prob_2_original = marginal_prob_2 if marginal_prob_2 >=1 else marginal_prob_2
#   ### use the prob_sub to calculate the correlation
#   marginal_prob_1_cal = prob_sub if marginal_prob_1 >=1 else marginal_prob_1
#   marginal_prob_2_cal = prob_sub if marginal_prob_2 >=1 else marginal_prob_2
#   var_1 = marginal_prob_1*(1-marginal_prob_1_cal)
#   var_2 = marginal_prob_2*(1-marginal_prob_2_cal)
#   either_default_prob = 1-(2*r0/(math.sqrt(2*math.pi*t)))*math.exp(-(r0**2/(4*t)))*inf_sum
#   default_corr = (marginal_prob_1_cal+marginal_prob_2_cal-either_default_prob-marginal_prob_1_cal*marginal_prob_2_cal)/(math.sqrt(var_1*var_2))
#   return default_corr,marginal_prob_1_original,marginal_prob_2_original


# def get_decorr_two_industries_zhou(start,end,industry1,industry2,prob_sub,show_non_normal = None):
#   companies = get_companies()
#   company_list1 = get_companies_from_industry(industry1)
#   company_list2 = get_companies_from_industry(industry2)
#   combination_list =  list(itertools.product(company_list1,company_list2))
#   print(len(company_list1),len(company_list2),len(combination_list))
#   num_pairs = len(combination_list)
#   corr_sum = 0
#   not_normal_sum = 0
#   non_normal_dic = {}
#   for (firm1,firm2) in combination_list:
#     result = analyse_two_companies_zhou(start,end,companies,firm1,firm2,prob_sub=0.9)
#     corr = result[0]
#     # print(corr_sum)
#     prob_default_1 = result[1]
#     prob_default_2 = result[2]
#     # print(corr,prob_default_1,prob_default_2)
#     # print(abs(corr)>1)
#     if (abs(corr) > 1):
#       num_pairs -= 1
#       corr_sum += 0
#       non_normal_dic[corr] = [prob_default_1,prob_default_2]
#     else:
#       corr_sum += corr
#   if show_non_normal == 'Yes':
#     return non_normal_dic
#   else:
#     return corr_sum/num_pairs

# def get_decorr_two_industries_zhou(start,end,industry1,industry2,prob_sub=0.9,show_non_normal = None):
#   companies = get_companies()
#   company_list1 = get_companies_from_industry(industry1)
#   company_list2 = get_companies_from_industry(industry2)
#   combination_list =  list(itertools.product(company_list1,company_list2))
#   print(len(company_list1),len(company_list2),len(combination_list))
#   num_pairs = len(combination_list)
#   corr_sum = 0
#   not_normal_sum = 0
#   non_normal_dic = {}
#   for (firm1,firm2) in combination_list:
#     result = analyse_two_companies_zhou(start,end,companies,firm1,firm2,prob_sub)
#     corr = result[0]
#     # print(corr_sum)
#     prob_default_1 = result[1]
#     prob_default_2 = result[2]
#     # print(corr,prob_default_1,prob_default_2)
#     # print(abs(corr)>1)
#     if (abs(corr) > 1):
#       num_pairs -= 1
#       corr_sum += 0
#       non_normal_dic[corr] = [prob_default_1,prob_default_2]
#     else:
#       corr_sum += corr
#   if show_non_normal == 'Yes':
#     return non_normal_dic
#   else:
#     return corr_sum/num_pairs

In [ ]:
# analyse_two_companies_zhou('2018-12-31','2023-12-31',get_companies(),get_companies()[3],get_companies()[4])[0]

In [ ]:
# industry_corr_matrix_5yr = np.zeros((len(get_industries()),len(get_industries())))
# for i in [i for i in range(0,len(get_companies()))]:
#   for j in [j for j in range(0,len(get_companies()))]:
#     if i == j:
#       industry_corr_matrix_5yr[i,j] = 1
#     else:
#       corr_i_j = get_decorr_two_industries_zhou('2018-12-31','2023-12-31',get_industries()[i],get_industries()[j],0.9)
#       industry_corr_matrix_5yr[i,j] = corr_i_j
# industry_corr_df_1 = pd.DataFrame(industry_corr_matrix_5yr)
# industry_corr_df_1

In [ ]:
# dic_1 = get_decorr_two_industries_zhou(get_date()[20],get_date()[24],get_industries()[4],get_industries()[10],0.9,'Yes')
# dic_2 = get_decorr_two_industries_zhou(get_date()[20],get_date()[24],get_industries()[4],get_industries()[10],0.95,'Yes')
# dic_3 = get_decorr_two_industries_zhou(get_date()[20],get_date()[24],get_industries()[4],get_industries()[10],0.99,'Yes')
# corr_list_1 = list(dic_1.keys())
# corr_list_2 = list(dic_2.keys())
# corr_list_3 = list(dic_3.keys())
# x1_list = [i for i in range(len(corr_list_1))]
# x2_list = [i for i in range(len(corr_list_2))]
# x3_list = [i for i in range(len(corr_list_3))]
# plt.plot(x1_list,corr_list_1,label = 'prob_sub = 0.9')
# plt.plot(x2_list,corr_list_2,label = 'prob_sub = 0.95' )
# plt.plot(x3_list,corr_list_3,label = 'prob_sub = 0.99')
# plt.ylabel('Default correlation')
# plt.xlabel('index')
# plt.legend()
# plt.show()

# Reproduce the result from Zhou(2001) paper

In [ ]:
# V1_0 = 1.8
# V2_0 = 1.8
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(0.2, 20, 0.5).tolist()
# rho_try_list = [0.1,0.2,0.4]
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_0,V2_0,k1,k2,sigma1,sigma2,-0.1,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_0,V2_0,k1,k2,sigma1,sigma2,0.2,t))
#  default_corr_list_3.append(get_default_corr_zhou(V1_0,V2_0,k1,k2,sigma1,sigma2,0.4,t))
# plt.plot(t_try_list,default_corr_list_1,label = 'rho = -0.1')
# # plt.plot(t_try_list,default_corr_list_2,label = 'rho = 0.2')
# plt.plot(t_try_list,default_corr_list_3,label = 'rho = 0.4')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.legend()
# plt.show()

In [ ]:
# bV1_01 = 2
# V2_01 = 2
# V1_02 = 2.5
# V2_02 = 2.5
# V1_03 = 3
# V2_03 = 3
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(0.5, 20.0, 0.5).tolist()
# rho_try_list = [0.1,0.2,0.4]
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = V2/K2 = 2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = V2/K2 = 2.5')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = V2/K2 = 3')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 1.2
# V2_01 = 1.2
# V1_02 = 1.5
# V2_02 = 1.5
# V1_03 = 2
# V2_03 = 2
# V1_04 = 7
# V2_04 = 7
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(0.5, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.1,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.1,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.1,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.1,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = V2/K2 = 1.5')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = V2/K2 = 2')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = V2/K2 = 7')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.1')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 1.2
# V2_01 = 1.2
# V1_02 = 1.5
# V2_02 = 1.5
# V1_03 = 2
# V2_03 = 2
# V1_04 = 7
# V2_04 = 7
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(0.5, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = V2/K2 = 1.5')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = V2/K2 = 2')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = V2/K2 = 7')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 1.2
# V2_01 = 1.2
# V1_02 = 1.5
# V2_02 = 1.5
# V1_03 = 2
# V2_03 = 2
# V1_04 = 7
# V2_04 = 7
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(0.5, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.9,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.9,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.9,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.9,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = V2/K2 = 1.5')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = V2/K2 = 2')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = V2/K2 = 7')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.9')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 1.2
# V2_01 = 1.2
# V1_02 = 1.5
# V2_02 = 1.5
# V1_03 = 2
# V2_03 = 2
# V1_04 = 7
# V2_04 = 7
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(0.5, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,-0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,-0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,-0.4,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,-0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = V2/K2 = 1.5')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = V2/K2 = 2')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = V2/K2 = 7')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = -0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 1.2
# V2_01 = 1.2
# V1_02 = 1.2
# V2_02 = 3
# V1_03 = 1.2
# V2_03 = 5
# V1_04 = 1.2
# V2_04 = 7
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(0.5, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = 1.2 V2/K2 = 3')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = 1.2 V2/K2 = 5')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = 1.2 V2/K2 = 7')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 0.2
# V2_01 = 0.2
# V1_02 = 0.3
# V2_02 = 0.3
# V1_03 = 0.8
# V2_03 = 0.8
# V1_04 = 0.95
# V2_04 = 0.95
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(0.5, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = V2/K2 = 0.1')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = V2/K2 = 0.3')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = V2/K2 = 0.8')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = V2/K2 = 0.95')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 0.2
# V2_01 = 1.2
# V1_02 = 0.3
# V2_02 = 1.2
# V1_03 = 0.8
# V2_03 = 1.2
# V1_04 = 0.95
# V2_04 = 1.2
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(1, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = 0.2, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = 0.3, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = 0.8 V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = 0.95 V2/K2 = 1.2')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 0.8
# V2_01 = 1.2
# V1_02 = 0.85
# V2_02 = 1.2
# V1_03 = 0.9
# V2_03 = 1.2
# V1_04 = 0.95
# V2_04 = 1.2
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(1, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = 0.8, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = 0.85, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = 0.9 V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = 0.95 V2/K2 = 1.2')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 0.8
# V2_01 = 1.05
# V1_02 = 0.85
# V2_02 = 1.05
# V1_03 = 0.9
# V2_03 = 1.05
# V1_04 = 0.95
# V2_04 = 1.05
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(1, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = 0.8, V2/K2 = 1.05')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = 0.85, V2/K2 = 1.05')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = 0.9 V2/K2 = 1.05')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = 0.95 V2/K2 = 1.05')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 0.2
# V2_01 = 1.05
# V1_02 = 0.4
# V2_02 = 1.05
# V1_03 = 0.6
# V2_03 = 1.05
# V1_04 = 0.9
# V2_04 = 1.05
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(1, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = 0.2, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = 0.4, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = 0.6 V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = 0.9 V2/K2 = 1.2')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 0.2
# V2_01 = 1.05
# V1_02 = 0.4
# V2_02 = 1.05
# V1_03 = 0.6
# V2_03 = 1.05
# V1_04 = 0.9
# V2_04 = 1.05
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(1, 15, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# default_corr_list_4 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
#  default_corr_list_4.append((get_default_corr_zhou(V1_04,V2_04,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = 0.2, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = 0.4, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = 0.6 V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_4,label = 'V1/K1 = 0.9 V2/K2 = 1.2')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# k1 = 1
# k2 = 1
# def get_v2_from_v1(v1,diff):
#   return v1+diff
# for i in rnage

In [ ]:
# V1_01 = 0.1
# V2_01 = 2
# V1_02 = 0.3
# V2_02 = 2
# V1_03 = 0.9
# V2_03 = 2
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(0.5, 20.0, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = 0.1, V2/K2 = 2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = 0.3, V2/K2 = 2')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = 0.9, V2/K2 = 2')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 0.1
# V2_01 = 2
# V1_02 = 0.3
# V2_02 = 2
# V1_03 = 0.9
# V2_03 = 2
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(1, 20.0, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = 0.1, V2/K2 = 2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = 0.3, V2/K2 = 2')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = 0.9, V2/K2 = 2')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

In [ ]:
# V1_01 = 0.1
# V2_01 = 1.2
# V1_02 = 0.3
# V2_02 = 1.2
# V1_03 = 0.9
# V2_03 = 1.2
# k1 = 1
# k2 = 1
# sigma1 = 0.4
# sigma2 = 0.4
# t_try_list = np.arange(1, 20.0, 0.5).tolist()
# default_corr_list_1 = []
# default_corr_list_2 = []
# default_corr_list_3 = []
# for t in  t_try_list:
#  default_corr_list_1.append(get_default_corr_zhou(V1_01,V2_01,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_2.append(get_default_corr_zhou(V1_02,V2_02,k1,k2,sigma1,sigma2,0.4,t))
#  default_corr_list_3.append((get_default_corr_zhou(V1_03,V2_03,k1,k2,sigma1,sigma2,0.4,t)))
# plt.plot(t_try_list,default_corr_list_1,label = 'V1/K1 = 0.1, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_2,label = 'V1/K1 = 0.3, V2/K2 = 1.2')
# plt.plot(t_try_list,default_corr_list_3,label = 'V1/K1 = 0.9, V2/K2 = 1.2')
# plt.ylabel('Default correlation')
# plt.xlabel('t')
# plt.title('sigma1 = sigma2 = 0.4,rho = 0.4')
# plt.legend()
# plt.show()

# Check the avilability of long term liability data

In [ ]:
# isin_list = ['CL0002694637', 'CL0000000035', 'MXP000511016', 'BRABEVACNOR1', 'MX01AM050019', 'BRAMERACNOR6', 'MX01AX040009', 'COB01PA00030', 'BRBBASACNOR3', 'BRBNBRACNOR8', 'PAP169941328', 'BRSANBACNPR5', 'CLP1506A1070', 'BRBRKMACNPA4', 'BRBRFSACNOR8', 'CLP256251073', 'MXP225611567', 'CLP3615W1037', 'BRCEEBACNOR3', 'BRSBSPACNOR5', 'BRCMIGACNPR3', 'BRCSRNACNOR0', 'BRCSNAACNOR6', 'CLP249051044', 'BRCSANACNOR6', 'BMG2519Y1084', 'MXP339881098', 'MXP369181377', 'CLP3697S1034', 'BREMBRACNOR4', 'CLP371151059', 'CL0000001314', 'CLP7847L1080', 'CLP371861061', 'CLP3710M1090', 'CL0001583070', 'CLP3880F1085', 'MXP320321310', 'MX01GC2M0006', 'BRGGBRACNPR8', 'BRGOLLACNPR4', 'MXP4948K1056', 'MXP495211262', 'COT13PA00086', 'MX01KU000012', 'MXP370841019', 'MXP791641105', 'MXP4987V1378', 'MX01ID000009', 'MXP554091415', 'BRMYPKACNOR7', 'MXP606941179', 'CL0000000423', 'BRRENTACNOR4', 'CLP969051075', 'CLP6877L1036', 'BROIBRACNPR8', 'MX01OR010004', 'ARP432631215', 'BRPETRACNPR6', 'CL0002455229', 'BRSUZBACNOR0', 'ARP9028N1016', 'ARP9308R1039', 'BRTUPYACNOR1', 'BRUGPAACNOR8', 'BRUSIMACNPA6', 'BRVALEACNOR0', 'ARP9897X1319']

In [ ]:
# len(isin_list)

In [ ]:
# balance_sheet = pd.read_csv('balance sheet.csv')

In [ ]:
# balance_sheet.head(1)

In [ ]:
# temp = balance_sheet[balance_sheet['isin'].isin(isin_list)]
# temp.shape

In [ ]:
# np.sum(temp.isnull(), axis = 0)

In [ ]:
# temp[temp['lltq'].isnull()]['datadate'].value_counts()

In [ ]:
# np.sum(temp[temp['lltq'].isnull()]['datadate'].value_counts())

In [ ]:
# temp_company = temp[temp['lltq'].isnull()]['isin'].value_counts()
# temp[temp['lltq'].isnull()]['isin'].value_counts()

In [ ]:
# np.sum(temp[temp['lltq'].isnull()]['isin'].value_counts())